## Tone generator

In [ ]:
"""Generation of pure tones."""

import numpy as np
import sounddevice as sd
import logging

samplerate = 44100


class ToneGen:
    def __init__(self, device, attack, release):
        if attack <= 0 or release <= 0:
            raise ValueError("attack and release values have to be positive")
        self._stream = sd.OutputStream(device=device,
                                       callback=self._callback, channels=2)
        self._attack = np.round(_sampling(attack / 1000)).astype(int)
        self._release = np.round(_sampling(release / 1000)).astype(int)
        self._channel = 0
        self._prev_gain = 0
        self._index = 0
        gain_value = 0
        frequency = 0
        slope = 0
        self._callback_parameters = gain_value, slope, frequency
        self._gain_value = gain_value
        self._callback_status = sd.CallbackFlags()
        self._stream.start()

    def _callback(self, outdata, frames, time, status):
        assert frames > 0
        self._callback_status |= status
        # This is thread-safe, see http://stackoverflow.com/a/17881014/500098:
        gain_value, slope, frequency = self._callback_parameters
        outdata.fill(0)
        a = np.arange(self._index, self._index + frames)
        r = np.arange(frames) * slope + self._prev_gain + slope
        assert slope != 0 or (gain_value == 0 and self._prev_gain == 0)
        if slope > 0:
            g = np.minimum(gain_value, r)
        else:
            g = np.maximum(gain_value, r)
        out_signal = g * np.sin(2 * np.pi * frequency * a / samplerate)
        outdata[:, self._channel] = out_signal
        self._index += frames
        self._prev_gain = g[-1]

    def start(self, frequency, db_gain, earside=None):
        if self._gain_value != 0:
            raise ValueError("gain_value must be zero before calling start function")
        if db_gain == -np.inf:
            raise ValueError("db_gain cannot have an infinite value")
        gain_value = _dbl(db_gain)
        slope = gain_value / self._attack
        self._gain_value = gain_value
        self._frequency = frequency
        self._callback_parameters = gain_value, slope, frequency
        if earside == 'left':
            self._channel = 0
        elif earside == 'right':
            self._channel = 1
        else:
            raise ValueError("'left' or 'right'?")

    def stop(self):
        if self._gain_value == 0:
            raise ValueError("gain_value must not be equal to zero before calling stop function")
        gain_value = 0
        slope = - self._gain_value / self._release
        self._gain_value = gain_value
        self._callback_parameters = gain_value, slope, self._frequency

    def close(self):

        if self._callback_status:
            logging.warning(str(self._callback_status))
        self._stream.stop()

    def __enter__(self):
        return self

    def __exit__(self, *args):
        self.close()


def _dbl(db_value):
    return 10 ** (db_value / 20)


def _sampling(seconds):
    return samplerate * seconds


## Responder

In [ ]:
"""The responder module processes the reaction to acoustic signals."""

from .pyxhook import pyxhook
import threading
import os
import time


class Responder:

    def __init__(self, tone_duration):
        os.system("stty -echo")
        self._timeout = tone_duration
        self._event1 = threading.Event()
        self._event1.set()
        self._event2 = threading.Event()
        self._event2.set()
        self._hookman = pyxhook.HookManager()
        self._hookman.KeyDown = self._kbevent_down
        self._hookman.KeyUp = self._kbevent_up
       
        self._hookman.start()

    def close(self):
        time.sleep(0.01)
        self._hookman.cancel()

    def click_down(self):
        if self._event1.is_set() and not self._event2.is_set():
            return True
        else:
            return False

    def click_up(self):
        if self._event2.is_set():
            return True
        else:
            return False

    def clear(self):
        self._event1.clear()
        self._event2.clear()
        
    def wait_for_arrow(self):
        self._event1.clear()
        self._event1.wait()
        return self._key

    def wait_for_click_up(self):
        self._event2.wait()

    def wait_for_click_down_and_up(self):
        self._event1.wait()
        self._event2.wait()

    def _kbevent_down(self, event):
        if event.MessageName == "key down"  and event.Key == "1":
            self._key = "1"
            self._event1.set()
        if event.MessageName == "key down"  and event.Key == "2":
            self._key = "2"
            self._event1.set()
        if event.MessageName == "key down"  and event.Key == "3":
            self._key = "3"
            self._event1.set()
        if event.MessageName == "key down"  and event.Key == "space":
            self._key = "space"
            self._event2.clear()
            self._event1.set()
                  
    def _kbevent_up(self, event):
        if event.MessageName == "key up" and event.Key == "space":
            self._key = "space"
            self._event2.set()

   
    def __exit__(self, *args):
        os.system("stty echo")
        self.close()

    def __enter__(self):
        return self


## Audiogram

In [ ]:
"""Audiogram."""

import numpy as np
import csv
import matplotlib.pyplot as plt


def set_audiogram_parameters(dBHL, freqs, conduction, masking, earside,
                             ax=None, **kwargs):
    """Set measuring points.

    Parameters
    ----------
    dBHL : array_like
          dB(HearingLevel)
    freqs : array_like
          Frequency Vector in Hz
    ax: plt.Ax, optional
          Matplotlib Ax to plot on
    earside: str, default='left'
          'left' or 'right' ear
    masking: bool, default=False
          Masked or unmasked hearing test
    conduction: str, default='air'
          Air conduction is 'air' and bone conduction is 'bone'
    Returns
    -------
    plt.Axis
        Matplotlib axis containing the plot.

    """
    if ax is None:
        ax = plt.gca()
    if freqs is None:
        freqs = [125,250,500,750,1000,1500,2000,3000,4000,6000,8000]
    xticks = np.arange(len(freqs))
    ax.set_xlabel('frequency (Hz)')
    ax.set_ylabel('Hearing level (dBHL)')
    ax.set_xlim([-0.5, xticks[-1] + 0.5])
    ax.set_ylim([-20, 120])
    plt.setp(ax, xticks=xticks, xticklabels=sorted(freqs))
    major_ticks = np.arange(-20, 120, 10)
    minor_ticks = np.arange(-20, 120, 5)
    ax.set_yticks(major_ticks)
    ax.set_yticks(minor_ticks, minor=True)
    ax.grid(which='both')
    ax.invert_yaxis()
    ax.tick_params(axis='x', labelsize=5.5)
    ax.tick_params(axis='y', labelsize=5.5)
    ax.set_aspect(0.9 / ax.get_data_ratio())
    ax.axhspan(-20,25,facecolor='white',alpha=0.3)
    ax.axhspan(25,40,facecolor='yellow',alpha=0.3)
    ax.axhspan(40,55,facecolor='cyan',alpha=0.3)
    ax.axhspan(55,70,facecolor='green',alpha=0.3)
    ax.axhspan(70,90,facecolor='magenta',alpha=0.3)
    ax.axhspan(90,120,facecolor='red',alpha=0.3)
    ax.text(6,22,'normal hearing',horizontalalignment='center',style='oblique',fontsize='9')
    ax.text(6,32,'mild hearing loss',horizontalalignment='center',style='oblique',fontsize='9')
    ax.text(6,47,'moderate hearing loss',horizontalalignment='center',style='oblique',fontsize='9')
    ax.text(6,62,'moderately severe hearing loss',horizontalalignment='center',style='oblique',fontsize='9')
    ax.text(6,80,'severe hearing loss',horizontalalignment='center',style='oblique',fontsize='9')
    ax.text(6,105,'profound hearing loss',horizontalalignment='center',style='oblique',fontsize='9')
    ax.set_title('{} ear'.format(earside))

    if earside == 'left':
        color = 'b'
        if conduction == 'air' and masking == 'off':
            marker = 'x'
        elif conduction == 'air' and masking == 'on':
                marker = 's'
        else:
            raise NameError("Conduction has to be 'air' or 'bone'")
    elif earside == 'right':
        color = 'r'
        if conduction == 'air' and masking == 'off':
            marker = 'o'
        elif conduction == 'air' and masking == 'on':
            marker = '^'
        else:
            raise NameError("Conduction has to be 'air' or 'bone'")
    elif not earside == 'right' or not earside == 'left':
        raise NameError("'left' or 'right'?")
    lines = ax.plot(dBHL, color=color, marker=marker, markersize=4,
                    markeredgewidth=1, fillstyle='none')
    gridlines = ax.get_xgridlines() + ax.get_ygridlines()
    for line in gridlines:
        line.set_linestyle('-')
    return lines


def make_audiogram(filename, results=None):

        if results is None:
            results = 'audiometer/results'
        data = _read_audiogram(filename)
        conduction = [option for cond, option, none in data
                      if cond == 'Conduction'][0]
        masking = [option for mask, option, none in data
                   if mask == 'Masking'][0]

        if 'right' in [side for frequency, level, side in data] and (
           'left' in [side for frequency, level, side in data]):
            f, (ax1, ax2) = plt.subplots(ncols=2)
        else:
            ax1, ax2 = None, None
            f = plt.figure()

        if 'right' in [side for frequency, level, side in data]:
            dBHL, freqs = _extract_parameters(data, 'right')
            set_audiogram_parameters(dBHL, freqs, conduction, masking,
                                     earside='right', ax=ax1)

        if 'left' in [side for frequency, level, side in data]:
            dBHL, freqs = _extract_parameters(data, 'left')
            set_audiogram_parameters(dBHL, freqs, conduction, masking,
                                     earside='left', ax=ax2)

        f.savefig('{}{}.pdf'.format(results, filename))


def _read_audiogram(filename):
    with open('audiometer/results/{}'.format(filename), 'r') as csvfile:
        reader = csv.reader(csvfile)
        data = [data for data in reader]
    return data


def _extract_parameters(data, earside):
    parameters = sorted((float(frequency), float(level)) for level, frequency, side
                        in data if side == earside)
    dBHL = [level for frequency, level in parameters]
    freqs = [int(frequency) for frequency, level in parameters]
    return dBHL, freqs


## Controller

In [ ]:
from audiometer import tone_generator
from audiometer import responder
import numpy as np
import argparse
import time
import os
import csv
import random


def setup():

    parser = argparse.ArgumentParser(fromfile_prefix_chars='@')
    parser.add_argument(
        "--device", help='How to select your soundcard is '
        'shown in http://python-sounddevice.readthedocs.org/en/0.3.3/'
        '#sounddevice.query_devices', type=int, default=None)
    parser.add_argument("--begin-familiarization", type=float, default=40,
                        help="in dBHL")
    parser.add_argument("--attack", type=float, default=30)
    parser.add_argument("--release", type=float, default=40)
    parser.add_argument(
        "--tone-duration", type=float, default=1, help='For more'
        'information on the tone duration have a look at '
        'ISO8253-1 ch. 6.2.1')
    parser.add_argument("--tolerance", type=float, default=1.5)
    parser.add_argument(
        "--pause-time", type=float, default=[1, 2], nargs=2, help="The pause "
        "time is calculated by an interval [a,b] randomly. It represents "
        "the total duration after the tone presentation. Please note, "
        "the pause time has to be greater than or equal to the tone duration")
    parser.add_argument("--earsides", type=str, nargs='+',
                        default=['right', 'left'], help="The first list item "
                        "represents the beginning earside. The second list "
                        "item represents the ending earside, consequently. "
                        "It is also possible to choose only one earside, "
                        "left or right")
    parser.add_argument("--minor-increment", type=float, default=5)
    parser.add_argument("--minor-decrement", type=float, default=5)
    parser.add_argument("--major-increment", type=float, default=10)
    parser.add_argument("--major-decrement", type=float, default=10)
    parser.add_argument("--fam-level", type=float, default=-40)
    parser.add_argument("--freqs", type=float, nargs='+', default=[1000, 1500,
                        2000, 3000, 4000, 6000, 8000, 750, 500, 250, 125],
                        help='The size '
                        'and number of frequencies are shown in'
                        'DIN60645-1 ch. 6.1.1. Their order'
                        'are described in ISO8253-1 ch. 6.1')
    parser.add_argument("--conduction", type=str, default='air', help="How "
                        "do you connect the headphones to the head? Choose "
                        " air or bone.")
    parser.add_argument("--masking", default='off')
    parser.add_argument("--results", type=str,
                        default='audiometer/results/')
    parser.add_argument("--filename", default='result_{}'.format(time.strftime(
                        '%Y-%m-%d_%H-%M-%S')) + '.csv')

    parser.add_argument("--carry-on", type=str)
    parser.add_argument("--logging", action='store_true')
    parser.add_argument("--calib125", default=[125, -81, 17])
    parser.add_argument("--calib250", default=[250, -92, 12])
    parser.add_argument("--calib500", default=[500, -80, -5])
    parser.add_argument("--calib750", default=[750, -85, -3])
    parser.add_argument("--calib1000", default=[1000, -84, -4])
    parser.add_argument("--calib1500", default=[1500, -82, -4])
    parser.add_argument("--calib2000", default=[2000, -90, 2])
    parser.add_argument("--calib3000", default=[3000, -94, 10])
    parser.add_argument("--calib4000", default=[4000, -91, 11])
    parser.add_argument("--calib6000", default=[6000, -70, -5])
    parser.add_argument("--calib8000", default=[8000, -76, 1])

    args = parser.parse_args()

    if not os.path.exists(args.results):
        os.makedirs(args.results)

    return args


class Controller:

    def __init__(self):

        self.setup = setup()

        if self.setup.carry_on:
            self.csvfile = open(os.path.join(self.setup.results,
                                             self.setup.carry_on), 'r+')
            reader = csv.reader(self.csvfile)
            for row in reader:
                pass
            last_freq = row[1]
            self.setup.freqs = self.setup.freqs[self.setup.freqs.index(
                                                  int(last_freq)) + 1:]
            self.setup.earsides[0] = row[2]
            self.writer = csv.writer(self.csvfile)
        else:
            self.csvfile = open(os.path.join(self.setup.results,
                                             self.setup.filename), 'w')
            self.writer = csv.writer(self.csvfile)
            self.writer.writerow(['Conduction', self.setup.conduction, None])
            self.writer.writerow(['Masking', self.setup.masking, None])
            self.writer.writerow(['Intensity(dBHL)', 'Frequency(Hz)', 'Earside'])

        self.cal_parameters = np.vstack((self.setup.calib125,
                                        self.setup.calib250,
                                        self.setup.calib500,
                                        self.setup.calib750,
                                        self.setup.calib1000,
                                        self.setup.calib1500,
                                        self.setup.calib2000,
                                        self.setup.calib3000,
                                        self.setup.calib4000,
                                        self.setup.calib6000,
                                        self.setup.calib8000))

        self._audio = tone_generator.ToneGen(self.setup.device,
                                                 self.setup.attack,
                                                 self.setup.release)
        self._rpd = responder.Responder(self.setup.tone_duration)

    def tap(self, frequency, dBHL_Level, earside):
        if self.dBHLtodBFS(frequency, dBHL_Level) > 0:
            raise OverflowError
        self._rpd.clear()
        self._audio.start(frequency, self.dBHLtodBFS(frequency, dBHL_Level),
                          earside)
        time.sleep(self.setup.tone_duration)
        click_down = self._rpd.click_down()
        self._audio.stop()
        if click_down:
            start = time.time()
            self._rpd.wait_for_click_up()
            End = time.time()
            if (End - start) <= self.setup.tolerance:
                time.sleep(random.uniform(self.setup.pause_time[0],
                           self.setup.pause_time[1]))
                return True
            else:
                time.sleep(random.uniform(self.setup.pause_time[0],
                           self.setup.pause_time[1]))
                return False

        else:
            time.sleep(random.uniform(self.setup.pause_time[0],
                                      self.setup.pause_time[1]))
            return False

    def cleartone(self, frequency, dBHL_Level, earside):
        self.key = ''
        while self.key != '3':
            if self.dBHLtodBFS(frequency, dBHL_Level) > 0:
                print("WARNING: Too loud. Signal distorted")
            self._audio.start(frequency,
                              self.dBHLtodBFS(frequency, dBHL_Level),
                              earside)
            self.key = self._rpd.wait_for_arrow()
            if self.key == '1':
                dBHL_Level -= 10
            if self.key == '2':
                dBHL_Level += 5
            self._audio.stop()

        return dBHL_Level

    def wait_for_click(self):
        self._rpd.clear()
        self._rpd.wait_for_click_down_and_up()
        time.sleep(1)

    def save(self, level, frequency, earside):
        row = [level, frequency, earside]
        self.writer.writerow(row)

    def dBHLtodBFS(self, frequency_value, dBHL):
        calibration = [(ref, corr) for frequency, ref, corr in self.cal_parameters
                       if frequency == frequency_value]
        return calibration[0][0] + calibration[0][1] + dBHL

    def __enter__(self):
        return self

    def __exit__(self, *args):
        time.sleep(0.1)
        self._rpd.__exit__()
        self._audio.close()
        self.csvfile.close()



# Main code Ascending method

In [1]:
#!/usr/bin/env python3
"""Ascending method.

For more details about the 'ascending method', have a look at
https://github.com/franzpl/audiometer/blob/master/docu/docu_audiometer.ipynb
The 'ascending method' is described in chapter 3.1.1

**WARNING**: If the hearing loss is too severe, this method will
not work! Please, consult an audiologist!

**WARNUNG**: Bei extremer SchwerhÃ¶rigkeit ist dieses Verfahren nicht
anwendbar! Bitte suchen Sie einen Audiologen auf!

"""

import sys
import logging
import time
from audiometer import controller
from audiometer import audiogram


logging.basicConfig(level=logging.DEBUG, format='%(levelname)s:%(message)s',
                    handlers=[logging.FileHandler("logfile.log", 'w'),
                              logging.StreamHandler()])


class AscendingMethod:



    print("\n\n\n\n*********************************************************************WELCOME!!*************************************************************************************")
    time.sleep(3)
    print("\n\nREAD THE INSTRUCTIONS CAREFULLY BEFORE THE TEST BEGINS\n\n1) To begin with increase or decrease the tone intensity with '2' and '1' NUM KEYS according to the hearing comfort\n2) Once the desired tone level is set press on the '3' num key to proceed for the test\n3) Press the space bar once to begin the test\n4) Kindly press and hold the space bar whenever the played tone is heard to you each time")
    
    time.sleep(10)
    

    def __init__(self):
        self.ctrl = controller.Controller()
        self.present_value = 0
        self.click = True

        print("\n\nREADY!!, click the space bar to begin")
        self.ctrl.wait_for_click()

    def decrement_value(self, decrement_level):

        self.present_value -= decrement_level
        self.click = self.ctrl.tap(self.frequency, self.present_value,
                                         self.earside)

    def increment_value(self, increment_level):

        self.present_value += increment_level
        self.click = self.ctrl.tap(self.frequency, self.present_value,
                                         self.earside)

    def familiarization(self):
        logging.info("Begin Familiarization")

        print("\nSet a clearly audible tone "
              "via the '1' and '2' num keys ('1' num key-decrease tone by 10dB, '2' num key- increase tone by 5dB) on the keyboard")
    
        time.sleep(3)

        print("Press the '3'num key once the tone is set")
        
        
        self.present_value = self.ctrl.cleartone(
                             self.frequency,
                             self.ctrl.setup.begin_familiarization,
                             self.earside)

        
        
    def test_begins(self):
        self.familiarization()

        print("\n\n\nTo begin the test, click space once\nPress and hold the space bar until the tone is heard")
        self.ctrl.wait_for_click()

        
        while self.click:
            logging.info("End Familiarization: -%s",
                         self.ctrl.setup.major_decrement)
            self.decrement_value(self.ctrl.setup.major_decrement)

        while not self.click:
            logging.info("+%s", self.ctrl.setup.minor_increment)
            self.increment_value(self.ctrl.setup.minor_increment)

        present_value_list = []
        present_value_list.append(self.present_value)

        two_answers = False
        while not two_answers:
            logging.info("2of3?: %s", present_value_list)
            for x in range(3):
                while self.click:
                    logging.info("-%s", self.ctrl.setup.minor_decrement)
                    self.decrement_value(
                        self.ctrl.setup.minor_decrement)

                while not self.click:
                    logging.info("+%s", self.ctrl.setup.minor_increment)
                    self.increment_value(
                        self.ctrl.setup.minor_increment)

                present_value_list.append(self.present_value)
                logging.info("2of3?: %s", present_value_list)
                # http://stackoverflow.com/a/11236055
                if [n for n in present_value_list
                   if present_value_list.count(n) == 2]:
                    two_answers = True
                    logging.info("2of3 --> True")
                    break
            else:
                logging.info("No Match! --> +%s",
                             self.ctrl.setup.major_increment)
                present_value_list = []
                self.increment_value(self.ctrl.setup.major_increment)

    def run(self):

        if not self.ctrl.setup.logging:
            logging.disable(logging.CRITICAL)

        for self.earside in self.ctrl.setup.earsides:
            for self.frequency in self.ctrl.setup.freqs:
                logging.info('freq:%s earside:%s', self.frequency, self.earside)
                try:
                    self.test_begins()
                    self.ctrl.save(self.present_value, self.frequency,
                                           self.earside)

                except OverflowError:
                    print("The signal is distorted, may be because of incorrect calibration or severe hearing loss.")
                    self.present_value = None
                    continue

                except KeyboardInterrupt:
                    sys.exit('\nInterrupted by user')

    def __enter__(self):
        return self

    def __exit__(self, *args):
        self.ctrl.__exit__()
        audiogram.make_audiogram(self.ctrl.setup.filename,
                                 self.ctrl.setup.results)

if __name__ == '__main__':

    with AscendingMethod() as asc_method:
        asc_method.run()
    print("\n\n\n***************************************************************************TEST COMPLETED***************************************************************************")
    time.sleep(10)




ModuleNotFoundError: No module named 'sounddevice'